# Acrobot

## A2C Agent 

In [6]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -445.0
Episode 7: -500.0
Episode 8: -379.0
Episode 9: -500.0
Episode 10: -301.0
Episode 11: -329.0
Episode 12: -253.0
Episode 13: -375.0
Episode 14: -265.0
Episode 15: -500.0
Episode 16: -407.0
Episode 17: -263.0
Episode 18: -299.0
Episode 19: -262.0
Episode 20: -450.0
Episode 21: -323.0
Episode 22: -175.0
Episode 23: -279.0
Episode 24: -254.0
Episode 25: -279.0
Episode 26: -176.0
Episode 27: -166.0
Episode 28: -248.0
Episode 29: -286.0
Episode 30: -182.0
Episode 31: -213.0
Episode 32: -166.0
Episode 33: -198.0
Episode 34: -133.0
Episode 35: -172.0
Episode 36: -180.0
Episode 37: -234.0
Episode 38: -180.0
Episode 39: -198.0
Episode 40: -141.0
Episode 41: -157.0
Episode 42: -145.0
Episode 43: -194.0
Episode 44: -144.0
Episode 45: -148.0
Episode 46: -148.0
Episode 47: -183.0
Episode 48: -98.0
Episode 49: -93.0
Episode 50: -180.0
Episode 51: -137.0
Episode 52: -133.0
Episo

In [7]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [1]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent
if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)

    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -398.3333333333333
Episode 8: -356.0
Episode 9: -283.6666666666667
Episode 10: -225.0
Episode 11: -242.66666666666666
Episode 12: -198.33333333333334
Episode 13: -252.0
Episode 14: -223.33333333333334
Episode 15: -227.0
Episode 16: -247.0
Episode 17: -172.0
Episode 18: -215.33333333333334
Episode 19: -191.33333333333334
Episode 20: -209.66666666666666
Episode 21: -303.3333333333333
Episode 22: -186.0
Episode 23: -210.33333333333334
Episode 24: -170.33333333333334
Episode 25: -217.0
Episode 26: -188.66666666666666
Episode 27: -210.33333333333334
Episode 28: -173.0
Episode 29: -174.0
Episode 30: -218.66666666666666
Episode 31: -208.33333333333334
Episode 32: -179.33333333333334
Episode 33: -157.66666666666666
Episode 34: -189.33333333333334
Episode 35: -153.66666666666666
Episode 36: -205.0
Episode 37: -204.66666666666666
Episode 38: -210.66666666666666

In [2]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [3]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    
agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration 
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2

    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)
        
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta += 0.1
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta)

    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -445.6666666666667
Episode 9: -497.3333333333333
Episode 10: -500.0
Episode 11: -500.0
Episode 12: -225.66666666666666
Episode 13: -273.0
Episode 14: -129.66666666666666
Episode 15: -175.66666666666666
Episode 16: -198.33333333333334
Episode 17: -145.0
Episode 18: -177.33333333333334
Episode 19: -174.33333333333334
Episode 20: -133.66666666666666
Episode 21: -165.66666666666666
Episode 22: -156.33333333333334
Episode 23: -214.33333333333334
Episode 24: -150.33333333333334
Episode 25: -120.66666666666667
Episode 26: -159.33333333333334
Episode 27: -135.0
Episode 28: -138.66666666666666
Episode 29: -119.0
Episode 30: -150.66666666666666
Episode 31: -147.66666666666666
Episode 32: -147.33333333333334
Episode 33: -130.0
Episode 34: -129.33333333333334
Episode 35: -125.33333333333333
Episode 36: -137.33333333333334
Episode 37: -233.0
Epis

Episode 290: -125.33333333333333
Episode 291: -153.0
Episode 292: -142.33333333333334
Episode 293: -135.0
Episode 294: -129.33333333333334
Episode 295: -134.66666666666666
Episode 296: -136.66666666666666
Episode 297: -110.0
Episode 298: -187.33333333333334
Episode 299: -132.66666666666666


In [4]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')